# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv", encoding="ISO-8859-1")

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secretsAPI import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 72912857, 'calculationPrice': 'close', 'change': 5.26, 'changePercent': 0.033, 'close': 168, 'closeSource': 'focfalii', 'closeTime': 1680633361096, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 169.6, 'delayedPriceTime': 1686008504853, 'extendedChange': -0.39, 'extendedChangePercent': -0.00237, 'extendedPrice': 167.55, 'extendedPriceTime': 1680369852405, 'high': 168, 'highSource': 'ee ii ycddpt15rmen luae', 'highTime': 1676949199748, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 168.71, 'iexCloseTime': 1721311425143, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 163.08, 'iexOpenTime': 1692494278626, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1719598076012, 'latestPrice': 165.05, 'latestSource': 'Close', 'latestTime': 'July 29, 2022', 'latestUpdate': 1664115896929, 'latestVolume': 103131583, 'low': 161.6, 'lowSource': 'dpn tedalmuie1c5eye  ri', '

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice']
marketCap = data['marketCap']
print(marketCap / 1e12)

2.707402848158


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ["Ticker", "Stock Price", "Market Cap", "# Shares To Buy"]
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Cap,# Shares To Buy


In [7]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        marketCap / 1e12,
        'N/A'
    ], index = my_columns), ignore_index=True
)

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\2512038652.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe.append(


,Ticker,Stock Price,Market Cap,# Shares To Buy
0,AAPL,165.05,2.707403,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'] / 1e12,
            'N/A'
        ],
        index = my_columns),
    ignore_index = True
    )

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\1319569913.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\1319569913.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\1319569913.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\1319569913.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\L

In [9]:
final_dataframe

,Ticker,Stock Price,Market Cap,# Shares To Buy
0,AAPL,167.20,2.744664,N/A
1,MSFT,280.81,2.139351,N/A
2,AMZN,136.82,1.389348,N/A
3,TSLA,926.22,0.958484,N/A
4,BRK.B,311.20,0.665885,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [11]:
symbol_strings = []
symbol_groups = list(chunks(stocks['Ticker'], 100))
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batchCall = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batchCall).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'] / 1e12,
                'N/A'
            ],
            index = my_columns),
            ignore_index = True
        )
final_dataframe

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppDa

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppDa

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppDa

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppDa

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppDa

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_28984\3797151645.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(


,Ticker,Stock Price,Market Cap,# Shares To Buy
0,AAPL,166.03,2.634359,N/A
1,MSFT,281.51,2.098992,N/A
2,AMZN,137.34,1.402118,N/A
3,TSLA,935.80,0.937464,N/A
4,BRK.B,304.80,0.689862,N/A
...,...,...,...,...
498,PENN,34.56,0.006032,N/A
499,VNO,31.29,0.006057,N/A
500,ALK,45.95,0.005600,N/A
501,PVH,62.92,0.004163,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [12]:
invalid_portfolio_size = True
while (invalid_portfolio_size):
    portfolio_size = input("Enter Portfolio Value: ")
    try:
        val = float(portfolio_size)
        invalid_portfolio_size = False
    except ValueError:
        print("Input must be a valid number")    

Enter Portfolio Value: 10000000


In [13]:
position_size = val / len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, "# Shares To Buy"] = math.floor(position_size / final_dataframe.loc[i, "Stock Price"])
final_dataframe
    

,Ticker,Stock Price,Market Cap,# Shares To Buy
0,AAPL,166.03,2.634359,119
1,MSFT,281.51,2.098992,70
2,AMZN,137.34,1.402118,144
3,TSLA,935.80,0.937464,21
4,BRK.B,304.80,0.689862,65
...,...,...,...,...
498,PENN,34.56,0.006032,575
499,VNO,31.29,0.006057,635
500,ALK,45.95,0.005600,432
501,PVH,62.92,0.004163,315


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [14]:
writer = pd.ExcelWriter("Recommended Trades.xlsx", engine="xlsxwriter")
final_dataframe.to_excel(writer, "Recommended Trades", index=False)


### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [15]:
background_color = '#0a0a23'
font_color = '#ffffff'
string_format = writer.book.add_format(
    {
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format':'$0.00',
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)
int_format = writer.book.add_format(
    {
        'num_format':'0',
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [16]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)

# writer.save()
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', dollar_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Cap', dollar_format)
# writer.sheets['Recommended Trades'].write('D1', 'Stocks To Buy', int_format)


This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [17]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Cap', dollar_format],
    'D': ['# Shares To Buy', int_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(column + '1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.save()